# Working with LUQ Datasets

Welcome to LUQ datasets tutorials. The aim of this tutorial is to provide guidance on creating your own datasets and use some of the pre-existing datasets.

## Step 1. Preprocess the dataset.

In order to unify the work with different QA datasets luq requires the user to pre-process the existing dataset into luq format. The required format is straightforward:

```json
{
    "data": [
        {
            "question": "Is this an example of a question?"‚
            "answer": "Yes, it is",
        },
        ...
    ]
}
```

There is a script that provide pre-processing for many existing datasets, such as CoQA, NQ, and others in `scripts/process_dataset.py`.

Let's for simplicity show it on CoQA processing example.

In [ ]:
!mkdir data/coqa
!python scripts/process_datasets.py \
    --dataset=coqa \
    --output=data/coqa/processed.json